<div class="alert alert-block" style="border: 2px solid #1976D2;background-color:#E3F2FD;padding:5px;font-size:0.9em;">
본 자료는 저작권법 제25조 2항에 의해 보호를 받습니다. 본 자료를 외부에 공개하지 말아주세요.<br>
본 강의만 잘 정리하면, 데이터 분석과 데이터 과학(머신러닝, 인공지능) 모두 가능합니다!<br>
<b><a href="https://school.fun-coding.org/">잔재미코딩</a> 에서 본 강의 기반 최적화된 로드맵도 확인하실 수 있습니다</b></div>

## 코로나 국가별 확진자 수 추이 분석/시각화
- https://public.flourish.studio/visualisation/2897018/

### 날짜별 확진자수 최종 데이터프레임 읽기 (df_confirmed)

In [112]:
import pandas as pd
df_confirmed = pd.read_csv("COVID-19-master/final_df.csv")
df_confirmed.head()

,Country_Region,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,...,2/28/2023,3/01/2023,3/02/2023,3/03/2023,3/04/2023,3/05/2023,3/06/2023,3/07/2023,3/08/2023,3/09/2023
0,Afghanistan,0,0,0,0,0,0,0,0,0,...,209322,209340,209358,209362,209369,209390,209406,209436,209451,209451
1,Albania,0,0,0,0,0,0,0,0,0,...,334391,334408,334408,334427,334427,334427,334427,334427,334443,334457
2,Algeria,0,0,0,0,0,0,0,0,0,...,271441,271448,271463,271469,271469,271477,271477,271490,271494,271496
3,Andorra,0,0,0,0,0,0,0,0,0,...,47866,47875,47875,47875,47875,47875,47875,47875,47890,47890
4,Angola,0,0,0,0,0,0,0,0,0,...,105255,105277,105277,105277,105277,105277,105277,105277,105288,105288


In [113]:
df_confirmed.shape

(201, 1144)

### 국가명과 iso2 매칭 테이블 읽기 (country_info)
  - iso2 컬럼값으로 https://www.countryflags.io/ 에서 제공하는 국기 이미지 링크를 얻을 수 있음
  - 다음 데이터는 결측치로 변환됨
    - ‘’, ‘#N/A’, ‘#N/A N/A’, ‘#NA’, ‘-1.#IND’, ‘-1.#QNAN’, ‘-NaN’, ‘-nan’, ‘1.#IND’, ‘1.#QNAN’, ‘<NA>’, ‘N/A’, ‘NA’, ‘NULL’, ‘NaN’, ‘n/a’, ‘nan’, ‘null’
  - Namibia 국가의 iso2 값이 NA 이므로 결측치 변환을 막기 위해 다음과 같은 옵션 설정
    - keep_default_na=False : 디폴트 결측치 변환 데이터를 사용하지 않고, na_values로 지정한 데이터만 결측치로 변환
    - na_values='' : 결측치로 변환할 값을 지정

In [114]:
country_info = pd.read_csv("COVID-19-master/csse_covid_19_data/UID_ISO_FIPS_LookUp_Table.csv", encoding='utf-8-sig', keep_default_na=False, na_values='')
country_info.head()

,Unnamed: 0.1,Unnamed: 0,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key
0,0,0,NaN,BW,NaN,NaN,NaN,NaN,NaN,Botswana,NaN,NaN,Botswana
1,1,1,NaN,BI,NaN,NaN,NaN,NaN,NaN,Burundi,NaN,NaN,Burundi
2,2,2,NaN,SL,NaN,NaN,NaN,NaN,NaN,Sierra Leone,NaN,NaN,Sierra Leone
3,3,3,4.0,AF,AFG,4.0,NaN,NaN,NaN,Afghanistan,33.93911,67.709953,Afghanistan
4,4,4,8.0,AL,ALB,8.0,NaN,NaN,NaN,Albania,41.15330,20.168300,Albania


In [115]:
country_info[country_info['Country_Region'] == 'Namibia']

,Unnamed: 0.1,Unnamed: 0,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key
115,115,115,516.0,NA,NAM,516.0,NaN,NaN,NaN,Namibia,-22.9576,18.4904,Namibia


In [116]:
country_info = country_info[['iso2', 'Country_Region']]
country_info.head()

,iso2,Country_Region
0,BW,Botswana
1,BI,Burundi
2,SL,Sierra Leone
3,AF,Afghanistan
4,AL,Albania


- 중복 행 제거

In [117]:
country_info.shape

(3560, 2)

In [118]:
country_info = country_info.drop_duplicates(subset='Country_Region', keep='first')
country_info.shape

(180, 2)

In [119]:
country_info

,iso2,Country_Region
0,BW,Botswana
1,BI,Burundi
2,SL,Sierra Leone
3,AF,Afghanistan
4,AL,Albania
...,...,...
175,ZW,Zimbabwe
199,AU,Australia
207,CA,Canada
222,CN,China


### 날짜별 국가별 확진자수와 국가별 iso2 값 병합

In [120]:
doc_final_country = pd.merge(df_confirmed, country_info, how='left', on='Country_Region')
doc_final_country.head()

,Country_Region,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,...,3/01/2023,3/02/2023,3/03/2023,3/04/2023,3/05/2023,3/06/2023,3/07/2023,3/08/2023,3/09/2023,iso2
0,Afghanistan,0,0,0,0,0,0,0,0,0,...,209340,209358,209362,209369,209390,209406,209436,209451,209451,AF
1,Albania,0,0,0,0,0,0,0,0,0,...,334408,334408,334427,334427,334427,334427,334427,334443,334457,AL
2,Algeria,0,0,0,0,0,0,0,0,0,...,271448,271463,271469,271469,271477,271477,271490,271494,271496,DZ
3,Andorra,0,0,0,0,0,0,0,0,0,...,47875,47875,47875,47875,47875,47875,47875,47890,47890,AD
4,Angola,0,0,0,0,0,0,0,0,0,...,105277,105277,105277,105277,105277,105277,105277,105288,105288,AO


In [121]:
doc_final_country.shape

(201, 1145)

#### 없는 데이터(NaN) 확인하기

In [122]:
doc_final_country.isnull().sum()

Country_Region     0
1/22/2020          0
1/23/2020          0
1/24/2020          0
1/25/2020          0
                  ..
3/06/2023          0
3/07/2023          0
3/08/2023          0
3/09/2023          0
iso2              23
Length: 1145, dtype: int64

### 특정 컬럼에 없는 데이터(NaN)가 있는 경우, 해당 행들만 보여주기
- 특정 컬럼 조건에 맞는 행들만 추출 가능
  - dataframe[dataframe[컬럼명] 조건]

In [123]:
doc_final_country[doc_final_country['iso2'].isnull()]

,Country_Region,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,...,3/01/2023,3/02/2023,3/03/2023,3/04/2023,3/05/2023,3/06/2023,3/07/2023,3/08/2023,3/09/2023,iso2
5,Antarctica,0,0,0,0,0,0,0,0,0,...,11,11,11,11,11,11,11,11,11,NaN
39,Comoros,0,0,0,0,0,0,0,0,0,...,9001,9001,9001,9001,9001,9001,9001,9008,9008,NaN
45,Cruise Ship,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
50,Diamond Princess,0,0,0,0,0,0,0,0,0,...,712,712,712,712,712,712,712,712,712,NaN
93,Kiribati,0,0,0,0,0,0,0,0,0,...,5013,5013,5013,5013,5013,5013,5013,5014,5014,NaN
94,"Korea, North",0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,NaN
102,Lesotho,0,0,0,0,0,0,0,0,0,...,34790,34790,34790,34790,34790,34790,34790,34790,34790,NaN
108,MS Zaandam,0,0,0,0,0,0,0,0,0,...,9,9,9,9,9,9,9,9,9,NaN
114,Marshall Islands,0,0,0,0,0,0,0,0,0,...,15618,15618,15618,15618,15618,15618,15618,15649,15649,NaN
118,Micronesia,0,0,0,0,0,0,0,0,0,...,23948,23948,23948,23948,23948,23948,23948,23948,23948,NaN


#### 참고: 복합 조건은 
- dataframe[(조건1) & (조건2)] : 조건1과 조건2 모두 만족 (and 조건)
- dataframe[(조건1) | (조건2)] : 조건1 또는 조건2 만족 (or 조건)
- 조건에 괄호를 넣는 것이 오동작을 방지하기 위해 좋음

```
nan_rows = doc_final_country[(doc_final_country['iso2'].isnull()) & (doc_final_country['4/01/2020'] == 0)]
nan_rows.head()
```

### 특정 컬럼에 없는 데이터 삭제하기

In [124]:
doc_final_country = doc_final_country.dropna(subset=['iso2'])

In [125]:
doc_final_country[doc_final_country['iso2'].isnull()]

,Country_Region,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,...,3/01/2023,3/02/2023,3/03/2023,3/04/2023,3/05/2023,3/06/2023,3/07/2023,3/08/2023,3/09/2023,iso2


<div class="alert alert-block" style="border: 1px solid #FFB300;background-color:#F9FBE7;">
<font size="4em" style="font-weight:bold;color:#BF360C;">2021년 업데이트 (국가 링크)</font><br>
<font size="3em" style=";color:#BF360C;">기존에는 www.countryflags.io 사이트를 활용하여, 국가 링크를 작성하는 부분을 설명드렸지만, 사이트이기 때문에, 해당 사이트 자체가 폐쇄되었습니다. 그래서, 급히 flagcdn.com 사이트를 기반으로 다음 코드를 업데이트하였으니, 코드 참고부탁드립니다.<br>이미지가 꼭 있어야 하는 부분은 아니라서, 해당 사이트도 폐쇄된다면, 국가 이미지 부분은 참고로만 보셔도 좋습니다.</font></div>

### 국기 링크를 기존 컬럼 기반해서 만들어, 데이터프레임에 붙이기 (2021 업데이트)
> 기존에는 영상에서처럼 다음 사이트를 활용하였으나, 해당 사이트 자체가 폐쇄되었습니다.
- 필요 데이터
  - 국가명, 국기, 날짜별 확진자 수
- 기존
~~- www.countryflags.io~~
  ~~- AD 는 https://www.countryflags.io/AD/flat/64.png~~
- 업데이트
  - AD 는 https://public.flourish.studio/country-flags/svg/ad.svg

- 국기 링크 만들기

In [126]:
iso2= 'AD'
# flag_link = 'https://flagcdn.com/48x36/' + iso2 + '.png'
# 2024 업데이트: iso2 를 소문자로 만들어야 해서 iso2.lower() 를 사용하였으며, 이미지 파일 확장자가 svg 이므로 svg 로 변경
# 추가로 https://public.flourish.studio/ 사이트에서 관련 이미지를 제공해주고 있으므로 다음과 같이 수정
flag_link = 'https://public.flourish.studio/country-flags/svg/' + iso2.lower() + '.svg'
flag_link

'https://public.flourish.studio/country-flags/svg/ad.svg'

#### 참고: 데이터프레임에 신규 컬럼 추가하기
- 데이터 프레임에 신규 컬럼 추가하기
  - 데이터프레임[신규컬럼] = pd.Series(data=데이터)
    - 데이터는 행에 맞게, [0, 1, 2] 와 같은 형태로 넣을 수 있음
- 기존 데이터프레임 컬럼 값을 기반으로 신규 컬럼 추가하기
  - apply() 함수를 사용
  - 다음 create_flag_link 함수 적용 전에, 아래의 함수로 어떤 값이 인자로 들어오는지 확인해보기 

    ```
    def create_flag_link(row):
        flag_link = 'https://flagcdn.com/48x36/' + row + '.png'
        return flag_link
        
    doc_final_country['Country_Flag'] = doc_final_country['iso2'].apply(create_flag_link)
    ```

In [127]:
def create_flag_link(row):
    # flag_link = 'https://flagcdn.com/48x36/' + row + '.png'
    # 2024 업데이트: iso2 를 소문자로 만들어야 해서 row.lower() 를 사용하였으며, 이미지 파일 확장자가 svg 이므로 svg 로 변경
    # 추가로 https://public.flourish.studio/ 사이트에서 관련 이미지를 제공해주고 있으므로 다음과 같이 수정
    flag_link = 'https://public.flourish.studio/country-flags/svg/' + row.lower() + '.svg'
    return flag_link

doc_final_country['iso2'] = doc_final_country['iso2'].apply(create_flag_link)

In [128]:
doc_final_country.head()

,Country_Region,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,...,3/01/2023,3/02/2023,3/03/2023,3/04/2023,3/05/2023,3/06/2023,3/07/2023,3/08/2023,3/09/2023,iso2
0,Afghanistan,0,0,0,0,0,0,0,0,0,...,209340,209358,209362,209369,209390,209406,209436,209451,209451,https://public.flourish.studio/country-flags/s...
1,Albania,0,0,0,0,0,0,0,0,0,...,334408,334408,334427,334427,334427,334427,334427,334443,334457,https://public.flourish.studio/country-flags/s...
2,Algeria,0,0,0,0,0,0,0,0,0,...,271448,271463,271469,271469,271477,271477,271490,271494,271496,https://public.flourish.studio/country-flags/s...
3,Andorra,0,0,0,0,0,0,0,0,0,...,47875,47875,47875,47875,47875,47875,47875,47890,47890,https://public.flourish.studio/country-flags/s...
4,Angola,0,0,0,0,0,0,0,0,0,...,105277,105277,105277,105277,105277,105277,105277,105288,105288,https://public.flourish.studio/country-flags/s...


### 데이터프레임 컬럼 조정하기

#### 1. 데이터프레임에서 필요한 컬럼만 선택하기
- 데이터프레임.columns.tolist() : 컬럼명을 리스트로 변환

In [129]:
cols = doc_final_country.columns.tolist()
cols

['Country_Region',
 '1/22/2020',
 '1/23/2020',
 '1/24/2020',
 '1/25/2020',
 '1/26/2020',
 '1/27/2020',
 '1/28/2020',
 '1/29/2020',
 '1/30/2020',
 '1/31/2020',
 '2/01/2020',
 '2/02/2020',
 '2/03/2020',
 '2/04/2020',
 '2/05/2020',
 '2/06/2020',
 '2/07/2020',
 '2/08/2020',
 '2/09/2020',
 '2/10/2020',
 '2/11/2020',
 '2/12/2020',
 '2/13/2020',
 '2/14/2020',
 '2/15/2020',
 '2/16/2020',
 '2/17/2020',
 '2/18/2020',
 '2/19/2020',
 '2/20/2020',
 '2/21/2020',
 '2/22/2020',
 '2/23/2020',
 '2/24/2020',
 '2/25/2020',
 '2/26/2020',
 '2/27/2020',
 '2/28/2020',
 '2/29/2020',
 '3/01/2020',
 '3/02/2020',
 '3/03/2020',
 '3/04/2020',
 '3/05/2020',
 '3/06/2020',
 '3/07/2020',
 '3/08/2020',
 '3/09/2020',
 '3/10/2020',
 '3/11/2020',
 '3/12/2020',
 '3/13/2020',
 '3/14/2020',
 '3/15/2020',
 '3/16/2020',
 '3/17/2020',
 '3/18/2020',
 '3/19/2020',
 '3/20/2020',
 '3/21/2020',
 '3/22/2020',
 '3/23/2020',
 '3/24/2020',
 '3/25/2020',
 '3/26/2020',
 '3/27/2020',
 '3/28/2020',
 '3/29/2020',
 '3/30/2020',
 '3/31/2020',
 

#### 데이터프레임 컬럼 위치 변경

In [130]:
cols.remove('iso2')
cols.insert(1, 'iso2')

In [131]:
doc_final_country = doc_final_country[cols]

In [132]:
doc_final_country.head()

,Country_Region,iso2,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,...,2/28/2023,3/01/2023,3/02/2023,3/03/2023,3/04/2023,3/05/2023,3/06/2023,3/07/2023,3/08/2023,3/09/2023
0,Afghanistan,https://public.flourish.studio/country-flags/s...,0,0,0,0,0,0,0,0,...,209322,209340,209358,209362,209369,209390,209406,209436,209451,209451
1,Albania,https://public.flourish.studio/country-flags/s...,0,0,0,0,0,0,0,0,...,334391,334408,334408,334427,334427,334427,334427,334427,334443,334457
2,Algeria,https://public.flourish.studio/country-flags/s...,0,0,0,0,0,0,0,0,...,271441,271448,271463,271469,271469,271477,271477,271490,271494,271496
3,Andorra,https://public.flourish.studio/country-flags/s...,0,0,0,0,0,0,0,0,...,47866,47875,47875,47875,47875,47875,47875,47875,47890,47890
4,Angola,https://public.flourish.studio/country-flags/s...,0,0,0,0,0,0,0,0,...,105255,105277,105277,105277,105277,105277,105277,105277,105288,105288


#### 컬럼명 변경

In [133]:
cols[1] = 'Country_Flag'

In [134]:
doc_final_country.columns = cols
doc_final_country.head()

,Country_Region,Country_Flag,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,...,2/28/2023,3/01/2023,3/02/2023,3/03/2023,3/04/2023,3/05/2023,3/06/2023,3/07/2023,3/08/2023,3/09/2023
0,Afghanistan,https://public.flourish.studio/country-flags/s...,0,0,0,0,0,0,0,0,...,209322,209340,209358,209362,209369,209390,209406,209436,209451,209451
1,Albania,https://public.flourish.studio/country-flags/s...,0,0,0,0,0,0,0,0,...,334391,334408,334408,334427,334427,334427,334427,334427,334443,334457
2,Algeria,https://public.flourish.studio/country-flags/s...,0,0,0,0,0,0,0,0,...,271441,271448,271463,271469,271469,271477,271477,271490,271494,271496
3,Andorra,https://public.flourish.studio/country-flags/s...,0,0,0,0,0,0,0,0,...,47866,47875,47875,47875,47875,47875,47875,47875,47890,47890
4,Angola,https://public.flourish.studio/country-flags/s...,0,0,0,0,0,0,0,0,...,105255,105277,105277,105277,105277,105277,105277,105277,105288,105288


### 최종 가공 완료 파일 저장

In [135]:
doc_final_country.to_csv("COVID-19-master/final_covid_data_for_graph.csv")

### 그래프 만들기
- https://app.flourish.studio/ 로그인
- New visualization 선택 -> Bar Chart Race 선택 -> 데이터 수정 (파일 업로드)

<div class="alert alert-block" style="border: 2px solid #1976D2;background-color:#E3F2FD;padding:5px;font-size:0.9em;">
본 자료는 저작권법 제25조 2항에 의해 보호를 받습니다. 본 자료를 외부에 공개하지 말아주세요.<br>
본 강의만 잘 정리하면, 데이터 분석과 데이터 과학(머신러닝, 인공지능) 모두 가능합니다!<br>
<b><a href="https://school.fun-coding.org/">잔재미코딩</a> 에서 본 강의 기반 최적화된 로드맵도 확인하실 수 있습니다</b></div>